### UQ in Geomagnetic Induction in Submarine Cable during the March 1989 Geomagnetic Storm
---
* __author__ = [Chakraborty, S.; Boteler, D. B. H.]
* __copyright__ = 
* __credits__ = []
* __license__ = MIT
* __version__ = 1.0
* __maintainer__ = Chakraborty, S.
* __email__ = shibaji7@vt.edu
* __status__ = Research

>  

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy.fftpack
from scipy.stats import pearsonr

import bezpy
from gic.model.oml import OceanModel
from gic.model.uqfy import QuantifyUnceartainity
from gic.model.utils import RecursiveNamespace

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use(["science", "ieee"])
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Tahoma", "DejaVu Sans",
                                   "Lucida Grande", "Verdana"]

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

%matplotlib inline

In [2]:
# Input Control JSON for Ocean-Earth model
ICJ = {
    "sid": 5,
    "opcode": 0,
    "Bfield": {
        "data_sources": { # Source of the datasets
            "stns": ["FRD", "HAD", "STJ"],
            "sources": [ # Linking data files to stations
                "input/data/1989/FRD_19890312_XYZ.txt",
                "input/data/1989/FRD_19890313_XYZ.txt",
                "input/data/1989/FRD_19890314_XYZ.txt",
                "input/data/1989/HAD_19890312_HDZ.txt",
                "input/data/1989/HAD_19890313_HDZ.txt",
                "input/data/1989/HAD_19890314_HDZ.txt",
                "input/data/1989/STJ_19890312_XYZ.txt",
                "input/data/1989/STJ_19890313_XYZ.txt",
                "input/data/1989/STJ_19890314_XYZ.txt",
            ],
        },
        "tapering": 0.1 # Tapering coefficient
    },
    "cable": { # Cable description
        "cable_sections": [ # Description of cable sections
            {
                "sec_id": 1,
                "station": "FRD",
                "elec_params": {
                    "earth_model": "CS-W", # CS-W Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 100, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 39.6, "lon": -74.33
                    },
                    "fin": { # Final lat/lon
                        "lat": 38.79, "lon": -72.62
                    }
                },
            },
            {
                "sec_id": 2,
                "station": "FRD",
                "elec_params": {
                    "earth_model": "DO-1", # DO-1 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 4000, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 38.79, "lon": -72.62
                    },
                    "fin": { # Final lat/lon
                        "lat": 37.11, "lon": -68.94
                    }
                },
            },
            {
                "sec_id": 3,
                "station": "STJ",
                "elec_params": {
                    "earth_model": "DO-2", # DO-2 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 5200, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 37.11, "lon": -68.94
                    },
                    "fin": { # Final lat/lon
                        "lat": 39.80, "lon": -48.20
                    }
                },
            },
            {
                "sec_id": 4,
                "station": "STJ",
                "elec_params": {
                    "earth_model": "DO-3", # DO-3 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 4000, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 39.80, "lon": -48.20
                    },
                    "fin": { # Final lat/lon
                        "lat": 40.81, "lon": -45.19
                    }
                },
            },
            {
                "sec_id": 5,
                "station": "STJ",
                "elec_params": {
                    "earth_model": "DO-4", # DO-4 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 4800, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 40.81, "lon": -45.19
                    },
                    "fin": { # Final lat/lon
                        "lat": 43.15, "lon": -39.16
                    }
                },
            },
            {
                "sec_id": 6,
                "station": "STJ",
                "elec_params": {
                    "earth_model": "DO-5", # DO-5 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 4000, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 43.15, "lon": -39.16
                    },
                    "fin": { # Final lat/lon
                        "lat": 44.83, "lon": -34.48
                    }
                },
            },
            {
                "sec_id": 7,
                "station": "STJ",
                "elec_params": {
                    "earth_model": "MAR", # MAR Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 3000, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 44.83, "lon": -34.48
                    },
                    "fin": { # Final lat/lon
                        "lat": 46.51, "lon": -22.43
                    }
                },
            },
            {
                "sec_id": 8,
                "station": "HAD",
                "elec_params": {
                    "earth_model": "DO-6", # DO-6 Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 4500, # Ocean depth in meters
                    "ocean_resistivity": 0.3 # Ocean resistivity
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 46.51, "lon": -22.43
                    },
                    "fin": { # Final lat/lon
                        "lat": 47.85, "lon": -9.05
                    }
                },
            },
            {
                "sec_id": 9,
                "station": "HAD",
                "elec_params": {
                    "earth_model": "CS-E", # CS-E Earth model stored in 'config/conductivity.json'
                    "ocean_depth": 100, # Ocean depth in meters
                    "ocean_resistivity": 0.3, # Ocean resistivity
                    "active_termination": True # Active termination on the East Edge
                },
                "edge_loc":{ # Cabel section edge location
                    "ini": { # Initial lat/lon
                        "lat": 47.85, "lon": -9.05
                    },
                    "fin": { # Final lat/lon
                        "lat": 50.79, "lon": -4.55
                    }
                },
            }
        ],
        "cable_pot_plot_index": 0
    },
    "out_dir": "tmp/out/006/", # Output directory
    "opname": "March 1989 Storm Analysis"
}

control = RecursiveNamespace(**ICJ)

In [3]:
qu = QuantifyUnceartainity(control)

2022-11-15 08:18:58.227 | INFO     | gic.model.uqfy:quntify_uncertainity_in_cable_sections:46 - Quantifying input unceratinity along cable routes
2022-11-15 08:18:58.228 | INFO     | gic.model.uqfy:quntify_uncertainity_in_cable_sections:51 - Estimating unceratinity along cable section 1
2022-11-15 08:18:58.230 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 40. -74.]
/home/shibaji/CodeBase/submarine_cable_modeling/gic/model/conductivity.py:307: RuntimeWarning: Mean of empty slice
  water_top = np.nanmean(water_top_values)
/home/shibaji/CodeBase/submarine_cable_modeling/gic/model/conductivity.py:308: RuntimeWarning: Mean of empty slice
  water_bot = np.nanmean(water_bot_values)
2022-11-15 08:18:58.243 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0         NaN          0.3
1    2.521000          3.0
2   32.521999       3000.0
3  133.609009       1000.0
4  241.382996        100.0
5  250.000000         10.0
6

2022-11-15 08:18:58.322 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 40. -71.]
2022-11-15 08:18:58.327 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    1.943000          0.3
1    8.052999          3.0
2    2.047000       3000.0
3  117.020004       1000.0
4  280.936996        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.327 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 40. -71.]
2022-11-15 08:18:58.332 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    1.943000          0.3
1    8.052999          3.0
2    2.047000       3000.0
3  117.020004       1000.0
4  280.936996        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.332 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 40. -70.]
2022-11-15 08:18:58.337 | INFO     | gic.model.condu

2022-11-15 08:18:58.418 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 40. -48.]
2022-11-15 08:18:58.422 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    5.303000          0.3
1    0.806000          3.0
2   10.092999       3000.0
3  182.885010       1000.0
4  210.912994        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.422 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 39. -48.]
2022-11-15 08:18:58.427 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    5.351000          0.3
1    0.649000          3.0
2    7.703000       3000.0
3  210.013000       1000.0
4  186.283997        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.427 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 39. -47.]
2022-11-15 08:18:58.432 | INFO     | gic.model.condu

2022-11-15 08:18:58.518 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    4.402000          0.3
1    0.425000          3.0
2    9.431000       3000.0
3  103.989998       1000.0
4  291.751999        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.519 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 37. -40.]
2022-11-15 08:18:58.524 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    4.038000          0.3
1    0.355000          3.0
2   10.979000       3000.0
3   79.413002       1000.0
4  315.214996        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.524 | INFO     | gic.model.uqfy:quntify_uncertainity_in_cable_sections:51 - Estimating unceratinity along cable section 6
2022-11-15 08:18:58.525 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 43. -39.]
2022-11-15 08:

2022-11-15 08:18:58.613 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    2.751000          0.3
1    0.067000          3.0
2    9.653000       3000.0
3   24.368999       1000.0
4  373.160000        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.614 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 48. -26.]
2022-11-15 08:18:58.618 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    2.742000          0.3
1    0.095000          3.0
2   10.365000       3000.0
3   26.460999       1000.0
4  370.337002        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.618 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [ 49. -24.]
2022-11-15 08:18:58.623 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    3.717000          0.3


2022-11-15 08:18:58.707 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    0.263000          0.3
1    3.389000          3.0
2   18.337000       3000.0
3   68.457001       1000.0
4  319.554001        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.708 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [49. -7.]
2022-11-15 08:18:58.712 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    0.166000          0.3
1    3.712000          3.0
2   16.643000       3000.0
3   95.069000       1000.0
4  294.410004        100.0
5  250.000000         10.0
6  340.000000          1.0
2022-11-15 08:18:58.713 | INFO     | gic.model.conductivity:compile_profiles:468 - Lat-lon: [49. -7.]
2022-11-15 08:18:58.717 | INFO     | gic.model.conductivity:compile_profiles:470 - Compiled Profile 
     thickness  resistivity
0    0.166000          0.3
1   